### SKELETON - THIS NOTEBOOK IS INTENDED TO BE GUTTED FOR A NEW PROJECT

In [3]:
%matplotlib inline
from pathlib import Path
import pickle
from PIL import Image
import io
import gdown
import zipfile

import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import argparse

# define seed
manualSeed = 999
print(f"Seed: {manualSeed}")
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Seed: 999


### Load Data

In [4]:
# training imgs download url + output file name definition
url = 'https://drive.google.com/uc?id=1A423Vi62SWb3FHtwieXDlmtIEcQHb2ub'
outfile = "imgs.zip"

# download imgs if imgs folder does not exist
if not os.path.exists("imgs"):
    gdown.download(url, outfile, quiet=False)

    with zipfile.ZipFile(outfile, 'r') as zip_ref:
        zip_ref.extractall()
    os.remove("imgs.zip")

### Globals

In [5]:
cwd = os.getcwd() # collect current working directory
dataroot = Path(f"{cwd}/imgs") # define data directory
workers = 2 # number of workers for dataloader
batch_size = 128 # batch size during training (64 + 64)
num_channels = 3 # number of channels in the training images. For color images this is 3 (RGB)
image_size = 64 # also the size of feature maps for the generator and discriminator
z_size = 100 # size of z latent vector (i.e. size of generator input)
num_epochs = 100 # number of training epochs
# num_epochs = 400 # number of training epochs
lr = 0.0002 # learning rate for optimizers
num_gpu = 2 # number of GPUs available. Use 0 for CPU mode.

### Define Transforms, Loaders, and preview Data

In [ ]:
# define transform
tfs = transforms.Compose([transforms.Resize(image_size),
                          transforms.CenterCrop(image_size),
                          transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) # range per channel = [-1,1]

# define the dataset
dataset = dset.ImageFolder(root=dataroot,transform=tfs)

# define the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# define available device
device = torch.device("cuda:0" if (torch.cuda.is_available() and num_gpu > 0) else "cpu")

# plot training image grid
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))